# TC 5033
## Deep Learning
## Fully Connected Deep Neural Networks

#### Activity 1b: Implementing a Fully Connected Network for Kaggle ASL Dataset

- Objective

The aim of this part of the activity is to apply your understanding of Fully Connected Networks by implementing a multilayer network for the [Kaggle ASL (American Sign Language) dataset](https://www.kaggle.com/datasets/grassknoted/asl-alphabet). While you have been provided with a complete solution for a Fully Connected Network using Numpy for the MNIST dataset, you are encouraged to try to come up with the solution.

- Instructions

    This activity requires submission in teams of 3 or 4 members. Submissions from smaller or larger teams will not be accepted unless prior approval has been granted (only due to exceptional circumstances). While teamwork is encouraged, each member is expected to contribute individually to the assignment. The final submission should feature the best arguments and solutions from each team member. Only one person per team needs to submit the completed work, but it is imperative that the names of all team members are listed in a Markdown cell at the very beginning of the notebook (either the first or second cell). Failure to include all team member names will result in the grade being awarded solely to the individual who submitted the assignment, with zero points given to other team members (no exceptions will be made to this rule).

    Load and Preprocess Data: You are provided a starter code to load the data. Be sure to understand the code.

    Review MNIST Notebook (Optional): Before diving into this activity, you have the option to revisit the MNIST example to refresh your understanding of how to build a Fully Connected Network using Numpy.

    Start Fresh: Although you can refer to the MNIST solution at any point, try to implement the network for the ASL dataset on your own. This will reinforce your learning and understanding of the architecture and mathematics involved.

    Implement Forward and Backward Pass: Write the code to perform the forward and backward passes, keeping in mind the specific challenges and characteristics of the ASL dataset.
    
     Design the Network: Create the architecture of the Fully Connected Network tailored for the ASL dataset. Choose the number of hidden layers, neurons, and hyperparameters judiciously.

    Train the Model: Execute the training loop, ensuring to track performance metrics such as loss and accuracy.

    Analyze and Document: Use Markdown cells to document in detail the choices you made in terms of architecture and hyperparameters, you may use figures, equations, etc to aid in your explanations. Include any metrics that help justify these choices and discuss the model's performance.  

- Evaluation Criteria

    - Code Readability and Comments
    - Appropriateness of chosen architecture and hyperparameters for the ASL dataset
    - Performance of the model on the ASL dataset (at least 70% acc)
    - Quality of Markdown documentation

- Submission

Submit this Jupyter Notebook in canvas with your complete solution, ensuring your code is well-commented and includes Markdown cells that explain your design choices, results, and any challenges you encountered.


In [2]:
#%pip install -q numpy
#%pip install -q pandas
#%pip install -q matplotlib

In [3]:
import os
import string

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

### Load Files

Load the csv files using pandas

In [4]:
train_df = pd.read_csv('../asl_data/sign_mnist_train.csv')
valid_df = pd.read_csv('../asl_data/sign_mnist_valid.csv')

In [5]:
train_df.sample(10)

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
26074,16,108,113,117,121,125,127,129,132,134,...,57,55,64,141,196,165,153,106,130,138
20744,7,209,211,214,216,216,219,219,220,221,...,244,243,242,242,241,240,239,237,237,236
13906,3,159,164,166,170,173,175,178,179,181,...,105,104,109,106,104,99,78,58,44,39
14028,16,138,141,146,151,156,159,161,165,167,...,96,209,212,211,210,210,209,208,208,206
19611,5,204,206,209,210,210,210,211,212,210,...,246,246,216,156,150,143,137,132,123,108
15716,6,183,185,186,186,186,186,186,186,185,...,209,208,208,207,205,203,202,201,198,196
15443,15,137,142,150,156,160,165,168,171,175,...,74,64,29,31,54,57,36,34,36,27
14379,15,135,140,147,154,158,162,164,168,172,...,76,68,31,28,38,40,41,59,53,31
4099,15,73,88,55,0,0,30,31,36,41,...,70,74,52,59,59,63,66,60,63,51
5976,7,55,70,83,96,107,115,117,120,127,...,70,71,71,72,69,71,142,171,151,141


In [6]:
array = train_df['label'].unique()

sorted(array)

[np.int64(0),
 np.int64(1),
 np.int64(2),
 np.int64(3),
 np.int64(4),
 np.int64(5),
 np.int64(6),
 np.int64(7),
 np.int64(8),
 np.int64(9),
 np.int64(10),
 np.int64(11),
 np.int64(12),
 np.int64(13),
 np.int64(14),
 np.int64(15),
 np.int64(16),
 np.int64(17),
 np.int64(18),
 np.int64(19),
 np.int64(20),
 np.int64(21),
 np.int64(22),
 np.int64(23)]

### Create Dataset

Creates the validation and test datasets

In [7]:
y_train = np.array(train_df['label'])
y_val = np.array(valid_df['label'])
del train_df['label']
del valid_df['label']
x_train = train_df.values.astype(np.float32)
x_val = valid_df.values.astype(np.float32)

In [8]:
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, shuffle=True, random_state=43)

x_val.shape, x_test.shape, y_val.shape, y_test.shape

((5379, 784), (1793, 784), (5379,), (1793,))

In [9]:
# Create targets to labels

alphabet=list(string.ascii_lowercase)

target2label = { i: l for i,l in enumerate(alphabet) }

num_classes = len(target2label.keys())

num_classes

26

# Data Exploration

In [11]:
fig, ax = plt.subplots(num_classes - 2, 10, figsize=(15, 25))

plt.tight_layout(pad=1.6)

for i, l in enumerate(alphabet):
    mask = y_train == i
    idx = np.where(mask == True)
    
    for j, id in enumerate(idx[0][:10]):
        ax[i,j].imshow(x_train[id, :].reshape((28,28)), cmap='gray')
        ax[i,j].set_axis_off()
        ax[i,j].title.set_text(f"letter: {l}")

### Normalization

Perform normalization on the dataset using the train mean and std. dev.

In [11]:
def normalise(x_mean, x_std, x_data):
    return (x_data - x_mean) / x_std

In [12]:
x_mean = x_train.mean()
x_std = x_train.std()

x_train = normalise(x_mean, x_std, x_train)
x_val = normalise(x_mean, x_std, x_val)
x_test = normalise(x_mean, x_std, x_test)

### Plot samples

In [13]:
def plot_image(image):
    plt.figure(figsize=(5,5))
    plt.imshow(np.reshape(image, (28, 28)), cmap=plt.get_cmap('gray'))
    plt.axis('off')
    plt.show()

In [ ]:
rnd_idx = np.random.randint(len(y_test))
print(f'The image is labeled with the label: {target2label[y_test[rnd_idx]]}')
plot_image(x_test[rnd_idx])

### Ecuaciones para nuestro modelo


$$z^1 = W^1 X + b^1$$

$$a^1 = ReLU(z^1) $$

$$z^2 = W^2 a^1 + b^2$$

$$\hat{y} = \frac{e^{z^{2_k}}}{\sum_j{e^{z_j}}}$$


$$ \mathcal{L}(\hat{y}^{i}, y^{i}) =  - y^{i}  \ln(\hat{y}^{i}) = -\ln(\hat{y}^i)$$


$$ \mathcal{J}(w, b) =  \frac{1}{num\_samples} \sum_{i=1}^{num\_samples}-\ln(\hat{y}^{i})$$

### Funciones adicionales

#### Mini batches

In [31]:
def create_minibatches(mb_size, x, y, shuffle = True):
    """This function creates mini-batches from the x and y datasets

    Args:
        mb_size (int): mini-batch size
        x (np.ndarray): train, valid or test dataset
        y (np.ndarray): train, valid or test labels
        shuffle (bool, optional): specifies if the batches are shuffled. Defaults to True.

    Returns:
        tuple: returns a tuple with the batches in pairs (x,y)
    """
    assert x.shape[0] == y.shape[0], 'Error en cantidad de muestras'
    total_data = x.shape[0]
    if shuffle: 
        idxs = np.arange(total_data)
        np.random.shuffle(idxs)
        x = x[idxs]
        y = y[idxs]  
    return ((x[i:i+mb_size], y[i:i+mb_size]) for i in range(0, total_data, mb_size))

## Nuestra clase Linear, ReLU y Sequential

In [32]:
class np_tensor(np.ndarray): pass

###  Clase Linear

In [33]:
class Linear:
    
    def __init__(self, input_size: int, output_size: int) -> None:
        """Fully Connected Layer

        Args:
            input_size (int): input size of the fully connected layer
            output_size (int): ouput size of the fully connected layer
        """
        self.W = (np.random.randn(output_size, input_size) / np.sqrt(input_size/2)).view(np_tensor)
        self.b = (np.zeros((output_size, 1))).view(np_tensor)
        
    def __call__(self, X):
        return self.W @ X + self.b
    
    def backward(self, X, Z):        
        X.grad = self.W.T @ Z.grad
        self.W.grad = Z.grad @ X.T
        self.b.grad = np.sum(Z.grad, axis = 1, keepdims=True)

### Clase ReLU

In [34]:
class ReLU:
    
    def __call__(self, Z):
        return np.maximum(0, Z)

    def backward(self, Z, A):
        Z.grad = A.grad.copy()
        Z.grad[Z <= 0] = 0

### Clase Sequential

In [41]:
class Sequential:

    def __init__(self, layers = None) -> None:
        """Creates a Neural Network of stacked layers

        Args:
            layers (list, optional): The list of layers. Defaults to None.
        """
        self.layers = layers
        self.outputs = {}

    def __call__(self, x):
        self.outputs['l0'] = x
        for i, layer in enumerate(self.layers, 1):
            x = layer(x)
            self.outputs['l' + str(i)] = x
        return x

    def backward(self):
        for i in reversed(range(len(self.layers))):
            li = self.outputs['l' + str(i)]
            li1 = self.outputs['l' + str(i + 1)]
            self.layers[i].backward(li, li1)

    def update(self, learning_rate = 1e-3):
        for layer in self.layers:
            if isinstance(layer, ReLU): continue
            layer.W = layer.W - learning_rate * layer.W.grad
            layer.b = layer.b - learning_rate * layer.b.grad

    def predict(self, x):
        return np.argmax(self.__call__(x))

### Cost Function

In [42]:
def softmaxXEntropy(x, y):
    batch_size = x.shape[1]
    exp_scores = np.exp(x)
    probs = exp_scores / exp_scores.sum(axis=0)
    preds = probs.copy()
    
    # Reshape de y para asegurar la forma correcta
    y = y.reshape(1, -1)
    
    # Cost
    y_hat = probs[y, np.arange(batch_size)]
    cost = np.sum(-np.log(y_hat)) / batch_size
    
    # Gradient calculation
    probs[y, np.arange(batch_size)] -= 1  # Corregido de y.squeeze() a y.reshape
    x.grad = probs.copy()
    
    return preds, cost

### Loop de entrenamiento

In [43]:
def train(model,epochs,mb_size=128, learning_rate=1e-3):
    for epoch in range(epochs):
        for i, (x,y) in enumerate(create_minibatches(mb_size, x_train,y_train)):
            scores= model(x.T.view(np_tensor))
            _, cost =softmaxXEntropy(scores,y)
            model.backward()
            model.update(learning_rate)
        print(f'cost: {cost}, accuracy: {accuracy(x_val,y_val,mb_size)}')
    

### Create your model and train it

In [44]:
def accuracy(x, y, mb_size):
    correct=0
    total= 0
    for i,(x,y) in enumerate(create_minibatches(mb_size,x,y)):
        pred= model(x.T.view(np_tensor))
        correct+= np.sum(np.argmax(pred,axis=0)== y.squeeze())
        total+= pred.shape[1]
    return correct/total


In [50]:
model = Sequential([Linear(784, 200), ReLU(), Linear(200, 200), ReLU(), Linear(200, num_classes)])
mb_size=512
learning_rate=1e-4
epochs=10

In [ ]:
train(model,epochs,mb_size,learning_rate)

In [ ]:
print(accuracy(x_test,y_test, mb_size))

### Test your model on Random data from your test set

In [ ]:
idx = np.random.randint(len(y_test))
plot_image(x_test[idx].reshape(28,28))
pred = model.predict(x_test[idx].reshape(-1, 1))
print(f'el valor predicho es: {target2label[pred]} el valor real es:{target2label[y_test[idx]]}')